In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
df = pd.read_csv('train.csv')

In [6]:
df.shape

(404290, 6)

In [7]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [12]:
#As the dataset is very big i need to select some less data.
new_df = df.sample(50000)

In [13]:
new_df.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [14]:
new_df.duplicated().sum()

np.int64(0)

In [15]:
ques_df = new_df[['question1','question2']]
ques_df.head()

,question1,question2
41479,How many people end up working in a field that...,Why do so many people with an degree in X end ...
94861,How do I reset my Outlook password?,How do I reset or recovery my Outlook password?
9826,What are the best places to visit in San Franc...,What are the best places to visit in San Franc...
236241,How do I purchase electronic items from Amazon...,I am moving from India to USA. Will my electro...
99582,Which courses are best for mechanical engineer...,After engineering which course is best to stud...


In [16]:
from sklearn.feature_extraction.text import CountVectorizer
#merge text
questions = list(ques_df['question1'])+list(ques_df['question2'])

cv = CountVectorizer(max_features=3000)
q1_arr,q2_arr=np.vsplit(cv.fit_transform(questions).toarray(),2) #splitted from middle to get q1_arr,q2_arr.

In [17]:
#Above a list of all the questions is created, top 3000 words are used to form bag_of_words.

In [18]:
temp_df1 = pd.DataFrame(q1_arr,index= ques_df.index)
temp_df2 = pd.DataFrame(q2_arr,index= ques_df.index)
temp_df = pd.concat([temp_df1,temp_df2],axis=1)
temp_df.shape

(50000, 6000)

In [19]:
temp_df.head()

,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
41479,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
94861,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9826,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
236241,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99582,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
temp_df['is_duplicate'] = new_df['is_duplicate']

In [21]:
temp_df.head()

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,is_duplicate
41479,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
94861,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9826,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
236241,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99582,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(temp_df.iloc[:,0:-1].values,temp_df.iloc[:,-1].values,test_size=0.2,random_state=42)

In [22]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
DTC = DecisionTreeClassifier()
DTC.fit(X_train,y_train)
y_pred = DTC.predict(X_test)
accuracy_score(y_test,y_pred)

0.6868

In [24]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
xgb=XGBClassifier()
xgb.fit(X_train,y_train)
y_pred = xgb.predict(X_test)
accuracy_score(y_test,y_pred)

0.7302